In [1]:
import pickle as pkl
import numpy as np
import wfdb
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
# pkl_file = open('signals_intervals.pkl', 'rb')
# data = pkl.load(pkl_file)

In [ ]:
len(data), len(data[0]), len(data[0][0])
# flat_list = [item for sublist in data for item in sublist]
# (flat_list[226]).shape
data[10][16][0].shape

In [22]:
def calc_next_len_conv1d(current_len=112500, kernel_size=16, stride=1, padding=0, dilation=1):
    return int(np.floor((current_len + 2*padding-dilation*(kernel_size-1)-1)/stride + 1))

len1 = calc_next_len_conv1d(current_len=248, kernel_size=5, stride=1, padding=2)
# len2 = calc_next_len_conv1d(current_len=len1, kernel_size=3, stride=1, padding=0)
# len2 = len2*2
# len3 = calc_next_len_conv1d(current_len=len2, kernel_size=11, stride=1)
# len4 = calc_next_len_conv1d(current_len=len3, kernel_size=13, stride=1)
# len4 = len4 * 2
# len5 = calc_next_len_conv1d(current_len=len4, kernel_size=3, stride=1)
# len6 = calc_next_len_conv1d(current_len=len5, kernel_size=5, stride=1)
# len6 = len6 * 2
# len7 = calc_next_len_conv1d(current_len=len6, kernel_size=3, stride=1)
# len7 = len7 * 2
# len8 = calc_next_len_conv1d(current_len=len7, kernel_size=3, stride=1)
len1

248

In [50]:
def calc_next_len_ConvTranspose1d(current_len=len3*4, kernel_size=16, stride=8, padding=0, dilation=1, out_padding=0):
    return int(stride*(current_len-1) - 2*padding+dilation*(kernel_size-1)+out_padding+1)

len_a = calc_next_len_ConvTranspose1d(current_len=1, kernel_size=64, stride=64, out_padding=1)
len_b = calc_next_len_ConvTranspose1d(current_len=len_a, kernel_size=48, stride=48, out_padding=5)
len_c = calc_next_len_ConvTranspose1d(current_len=len_b, kernel_size=6, stride=12, out_padding=6)
len_d = calc_next_len_ConvTranspose1d(current_len=len_c, kernel_size=3, stride=3)
len_d

112500

In [9]:
segment_len=2000
num_segments_in_record = 100

basic_arr_path = "data/mit-bih-arrhythmia-database-1.0.0" # where you downloaded mit-bih-arrhythmia db
arr_db = wfdb.get_record_list('mitdb')
signals = np.zeros([segment_len, num_segments_in_record, len(arr_db)])
sig_list=[]
for rec_id, record in enumerate(arr_db):
    record_path = os.path.join(basic_arr_path, str(record))
    record_signal = wfdb.rdsamp(record_path, sampto=num_segments_in_record*segment_len)[0][:, 0]
    sig_list.append(record_signal.reshape([num_segments_in_record, segment_len]))
signals = np.vstack(sig_list)

In [112]:
signals = np.vstack(listt)
# listt[1].shape
signals.shape

(4800, 2000)

In [40]:
def create_index_df(desired_segment_len=3600, basic_arr_path = "data/mit-bih-arrhythmia-database-1.0.0"):
    desired_segment_len=3600
    basic_arr_path = "data/mit-bih-arrhythmia-database-1.0.0"
    arr_db = wfdb.get_record_list('mitdb')
    num_samples_in_record = 30*60*360
    segment_dict = defaultdict(list)
    segment_list = list()
    for _, record_id in enumerate(arr_db):
        record_path = os.path.join(basic_arr_path, str(record_id))

        ann = wfdb.rdann(record_path, 'atr', sampto=num_samples_in_record, return_label_elements=['description'])
        df = pd.DataFrame({'description':ann.description, 'sample':ann.sample})    
        counter = 0
        reset_flag = True

        for i in range(1,df.shape[0]-1):
            curr_label, curr_sample = df.loc[i,['description','sample']]
            if reset_flag:
                start_sample = curr_sample
            next_label, next_sample = df.loc[i+1,['description','sample']]
            if curr_label == next_label:
                counter += next_sample-curr_sample
                reset_flag = False
                if counter > desired_segment_len:
                    segment_dict['record_id'].extend([record_id])
                    segment_dict['start_sample'].extend([start_sample])
                    segment_dict['label'].extend([curr_label])
                    counter = 0
                    reset_flag = True
                    signal = wfdb.rdsamp(record_path, sampfrom=start_sample, sampto=start_sample+3600)[0][:, 0]
                    segment_list.append(Segment(signal, curr_label))
            else:
                counter = 0
                reset_flag = True

    #     return pd.DataFrame(segment_dict)
    return segment_list

In [41]:
class Segment:
    def __init__(self, signal=1, label=1):
        self.signal = signal
        self.label = label

segment_list = create_index_df()

In [45]:
segment_list[1000].label

'Normal beat'